In [1]:
import git
import requests
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
git_paths = {'PyNWB': "NeurodataWithoutBorders/pynwb", 
             'MatNWB': "NeurodataWithoutBorders/matnwb",
             'NWBWidgets': "NeurodataWithoutBorders/nwb-jupyter-widgets",
             'NWBInspector': "NeurodataWithoutBorders/nwbinspector",
             'Hackathons': "NeurodataWithoutBorders/nwb_hackathons",
             'NWB_Schema': "NeurodataWithoutBorders/nwb-schema",
             'NWB_Schema_Language': "NeurodataWithoutBorders/nwb-schema-language",
             'HDMF': 'hdmf-dev/hdmf',
             'HDMF_Common_Schema': 'hdmf-dev/hdmf-common-schema',
             'HDMF_DocUtils': 'hdmf-dev/hdmf-docutils',
             'HDMF Schema Language' : 'hdmf-dev/hdmf-schema-language',
             'NDX_Template': 'nwb-extensions/ndx-template',
             'NDX_Staged_Extensions': 'nwb-extensions/staged-extensions',
             #'NDX Webservices': 'https://github.com/nwb-extensions/nwb-extensions-webservices.git',
             'NDX_Catalog': 'nwb-extensions/nwb-extensions.github.io',
             'NDX_Extension_Smithy': 'nwb-extensions/nwb-extensions-smithy',
             'NWB_1.x_Matlab': 'NeurodataWithoutBorders/api-matlab',
             'NWB_1.x_Python': 'NeurodataWithoutBorders/api-python'
            }

In [ ]:
def get_releases(repo_name):
    """
    Get release for the given repo
    
    :param repo_name: Name of the repo consisting of <owner>/<repo>, e.g., 'NeurodataWithoutBorders/pynwb'
    
    :returns: List of dicts with the release data
    """
    r = requests.get("https://api.github.com/repos/%s/releases" % (repo_name.lstrip('/').rstrip('/')))
    if not r.ok:
        r.raise_for_status()
    return r.json()

def get_release_names_and_dates(releases):
    """Given the output of get_releases get just the names and dates of releases"""
    names = []
    dates = []
    for rel in releases:
        if 'Latest' not in rel['name']:
            names.append(rel['tag_name'])
            dates.append(rel['published_at'])  
    dates = [datetime.strptime(d[0:10], "%Y-%m-%d") for d in dates]
    return names, dates

def plot_release_timeline(reponame, names, dates, figsize=None, tag_fontsize=14):
    # Based on https://matplotlib.org/stable/gallery/lines_bars_and_markers/timeline.html
    # Choose some nice levels
    levels = np.tile([-5, 5, -3, 3, -1, 1],
                     int(np.ceil(len(dates)/6)))[:len(dates)]

    # Create figure and plot a stem plot with the date
    fig, ax = plt.subplots(figsize=(12,5) if figsize is None else figsize)
    ax.set(title="%s release dates" % reponame)

    ax.vlines(dates, 0, levels, color="tab:blue")  # The vertical stems.
    ax.plot(dates, np.zeros_like(dates), "-o",
            color="k", markerfacecolor="w")  # Baseline and markers on it.

    # annotate lines
    for d, l, r in zip(dates, levels, names):
        ax.annotate(r, xy=(d, l),
                    xytext=(tag_fontsize+2, np.sign(l)*3), textcoords="offset points",
                    horizontalalignment="right",
                    verticalalignment="bottom" if l > 0 else "top",
                    fontsize=tag_fontsize)

    # format xaxis with 4 month intervals
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    # remove y axis and spines
    ax.yaxis.set_visible(False)
    #ax.spines[["left", "top", "right"]].set_visible(False)

    ax.margins(y=0.1)
    plt.tight_layout()
    plt.show()

In [19]:
for repo in ['PyNWB', 'MatNWB', 'HDMF', 'NWB_Schema', 'HDMF_Common_Schema']:
    releases = get_releases(git_paths[repo])
    release_names, release_dates = get_release_names_and_dates(releases)
    plot_release_timeline(repo, release_names, release_dates)

HTTPError: 403 Client Error: rate limit exceeded for url: https://api.github.com/repos/NeurodataWithoutBorders/pynwb/releases